# TPE analysis and Tissue detection


In this section we will look into some of the new datasets. In these dataset, there are typically four tissues for each recording. This means that in order to analyze this data, we need to extract traces from each tissue seprately.

First we make the necessary imports

In [ ]:
from pathlib import Path # For working with file paths
import mps  # For reading MPS data
import matplotlib.pyplot as plt # For plotting
import numpy as np  # For general numerical functions
import ap_features as apf

Next, we download the TPE data

In [ ]:
tpe_data_path = Path("tpe.npy")

def download_data():
    print("Downloading data. Please wait...")
    link = "https://www.dropbox.com/s/qga5fi6xzt306f1/tpe.npy?dl=1"
    import urllib.request
    import time

    urllib.request.urlretrieve(link, motion_data_path)
    time.sleep(1.0)
    print("Done downloading data")
    
if not tpe_data_path.is_file():
    download_data()
    
data = mps.MPS(tpe_data_path)

And just print some info about the dataset

In [ ]:
data.info

Let us plot the first frame in the dataset

In [ ]:
first_frame = data.frames[:, :, 0].T

In [ ]:
fig, ax = plt.subplots(figsize=(6, 10))
ax.imshow(first_frame)

We see that there are four tissues, where the tissue on the top is little bit more difficult to see (the pixel intensities are lower), but this tissue has one very bright region.

We can try to manually create some bounding boxes around these tissues.

In [ ]:
x0 = (50, 600)
y0 = (50, 200)

x1 = (50, 600)
y1 = (220, 350)

x2 = (50, 600)
y2 = (400, 520)

x3 = (50, 600)
y3 = (550, 700)

fig, ax = plt.subplots()
ax.imshow(first_frame)
for (xi, yi) in [(x0, y0), (x1, y1), (x2, y2), (x3, y3)]:
    ax.plot([xi[0], xi[0]], yi, color="r")
    ax.plot([xi[1], xi[1]], yi, color="r")
    ax.plot(xi, [yi[0], yi[0]], color="r")
    ax.plot(xi, [yi[1], yi[1]], color="r")

Now we can slice the images so that we get one set of images for each tissue specified by the bounding boxes defined above. Let us also plot the first frame of the second tissue to see that we actally got a reasonanle image

In [ ]:
tissue0 = data.frames[x0[0]:x0[1], y0[0]:y0[1], :]
tissue1 = data.frames[x1[0]:x1[1], y1[0]:y1[1], :]
tissue2 = data.frames[x2[0]:x2[1], y2[0]:y2[1], :]
tissue3 = data.frames[x3[0]:x3[1], y3[0]:y3[1], :]

fig, ax = plt.subplots()
ax.imshow(tissue1[:, :, 0].T)
plt.show()

For each tissue we can now overage over each image, so that each image is reduced to a single number. This will then result in a trace for each tissue which we can plot

In [ ]:
y_tissue0 = tissue0.mean(axis=(0, 1))
y_tissue1 = tissue1.mean(axis=(0, 1))
y_tissue2 = tissue2.mean(axis=(0, 1))
y_tissue3 = tissue3.mean(axis=(0, 1))

fig, ax = plt.subplots()
ax.plot(data.time_stamps, y_tissue0, label="tissue0")
ax.plot(data.time_stamps, y_tissue1, label="tissue1")
ax.plot(data.time_stamps, y_tissue2, label="tissue2")
ax.plot(data.time_stamps, y_tissue3, label="tissue3")
ax.legend()
plt.show()

We see that the baseline is quite different, and we can correct this baseline using a background correction method

In [ ]:
trace_tissue0 = apf.Beats(y=y_tissue0, t=data.time_stamps, background_correction_method="full")
trace_tissue1 = apf.Beats(y=y_tissue1, t=data.time_stamps, background_correction_method="full")
trace_tissue2 = apf.Beats(y=y_tissue2, t=data.time_stamps, background_correction_method="full")
trace_tissue3 = apf.Beats(y=y_tissue3, t=data.time_stamps, background_correction_method="full")

fig, ax = plt.subplots()
ax.plot(trace_tissue0.t, trace_tissue0.y, label="tissue0")
ax.plot(trace_tissue1.t, trace_tissue1.y, label="tissue1")
ax.plot(trace_tissue2.t, trace_tissue2.y, label="tissue2")
ax.plot(trace_tissue3.t, trace_tissue3.y, label="tissue3")
ax.legend()
plt.show()

We see now that the three last tissues are very similar, but the first tissue looks very different.

To improve the results futher, we can try to detect the tissues using some very backing image processing. Here we first blur the image using a gaussian filter with a standard deviation of 20.0. Then we remove the small objects less that 400 pixels inside. We could also fill the holes but in this example we decide not to. Try to experiment with different values of these arguments to see if it changes the result.

In [ ]:
mask = mps.tissue_detection.detect_tissue(frame=first_frame, sigma=20.0, min_size=400, fill_holes=False)

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
ax[0].imshow(first_frame)
ax[0].imshow(mask, alpha=0.1, cmap="gray")
ax[1].imshow(mask, cmap="gray")

We can now create a mask for each tissue

In [ ]:
mask_tissue0 = mask.T[x0[0]:x0[1], y0[0]:y0[1]]
mask_tissue1 = mask.T[x1[0]:x1[1], y1[0]:y1[1]]
mask_tissue2 = mask.T[x2[0]:x2[1], y2[0]:y2[1]]
mask_tissue3 = mask.T[x3[0]:x3[1], y3[0]:y3[1]]

fig, ax = plt.subplots()
ax.imshow(mask_tissue1.T)
plt.show()

And use this mask to compute new average traces, by onle averaging over the pixel inside the masked region

In [ ]:
y_tissue0_mask = tissue0[mask_tissue0].mean(axis=0)
y_tissue1_mask = tissue1[mask_tissue1].mean(axis=0)
y_tissue2_mask = tissue2[mask_tissue2].mean(axis=0)
y_tissue3_mask = tissue3[mask_tissue3].mean(axis=0)

fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)
ax[0].plot(data.time_stamps, y_tissue0_mask, label="tissue0")
ax[0].plot(data.time_stamps, y_tissue1_mask, label="tissue1")
ax[0].plot(data.time_stamps, y_tissue2_mask, label="tissue2")
ax[0].plot(data.time_stamps, y_tissue3_mask, label="tissue3")
ax[0].legend()
ax[0].set_title("Masked")
ax[1].plot(data.time_stamps, y_tissue0, label="tissue0")
ax[1].plot(data.time_stamps, y_tissue1, label="tissue1")
ax[1].plot(data.time_stamps, y_tissue2, label="tissue2")
ax[1].plot(data.time_stamps, y_tissue3, label="tissue3")
ax[1].legend()
ax[1].set_title("Original")
plt.show()

We see that the traces obtained using the masks have a higher baseline intensity. We can try to apply some baseline corrrection to compare the traces further

In [ ]:
trace_tissue0_mask = apf.Beats(y=y_tissue0_mask, t=data.time_stamps, background_correction_method="full")
trace_tissue1_mask = apf.Beats(y=y_tissue1_mask, t=data.time_stamps, background_correction_method="full")
trace_tissue2_mask = apf.Beats(y=y_tissue2_mask, t=data.time_stamps, background_correction_method="full")
trace_tissue3_mask = apf.Beats(y=y_tissue3_mask, t=data.time_stamps, background_correction_method="full")

fig, ax = plt.subplots(1, 2, figsize=(10, 6), sharey=True)
ax[0].plot(trace_tissue0_mask.t, trace_tissue0_mask.y, label="tissue0")
ax[0].plot(trace_tissue1_mask.t, trace_tissue1_mask.y, label="tissue1")
ax[0].plot(trace_tissue2_mask.t, trace_tissue2_mask.y, label="tissue2")
ax[0].plot(trace_tissue3_mask.t, trace_tissue3_mask.y, label="tissue3")
ax[0].legend()
ax[0].set_title("Masked")
ax[1].plot(trace_tissue0.t, trace_tissue0.y, label="tissue0")
ax[1].plot(trace_tissue1.t, trace_tissue1.y, label="tissue1")
ax[1].plot(trace_tissue2.t, trace_tissue2.y, label="tissue2")
ax[1].plot(trace_tissue3.t, trace_tissue3.y, label="tissue3")
ax[1].legend()
ax[1].set_title("Original")
plt.show()

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(10, 6), sharey="row", sharex=True)
ax[0, 0].plot(trace_tissue0_mask.t, trace_tissue0_mask.y)
ax[0, 0].set_ylabel("tissue0")
ax[1, 0].plot(trace_tissue1_mask.t, trace_tissue1_mask.y)
ax[1, 0].set_ylabel("tissue1")
ax[2, 0].plot(trace_tissue2_mask.t, trace_tissue2_mask.y)
ax[2, 0].set_ylabel("tissue2")
ax[3, 0].plot(trace_tissue3_mask.t, trace_tissue3_mask.y)
ax[3, 0].set_ylabel("tissue3")

ax[0, 1].plot(trace_tissue0.t, trace_tissue0.y)
ax[1, 1].plot(trace_tissue1.t, trace_tissue1.y)
ax[2, 1].plot(trace_tissue2.t, trace_tissue2.y)
ax[3, 1].plot(trace_tissue3.t, trace_tissue3.y)

ax[0, 0].set_title("Masked")
ax[0, 1].set_title("Original")

plt.show()

In [ ]:
def normalize(y):
    return (y - np.min(y)) / (np.max(y) - np.min(y))

# def normalize(y): return y

s = 1 # First point messes up the normalization
fig, ax = plt.subplots(4, 1, figsize=(10, 6), sharey="row", sharex=True)
l1, = ax[0].plot(trace_tissue0_mask.t[s:], normalize(trace_tissue0_mask.y[s:]))
l2, = ax[0].plot(trace_tissue0.t[s:], normalize(trace_tissue0.y[s:]))
ax[0].set_ylabel("tissue0")

ax[1].plot(trace_tissue1_mask.t, normalize(trace_tissue1_mask.y))
ax[1].plot(trace_tissue1.t, normalize(trace_tissue1.y))
ax[1].set_ylabel("tissue1")

ax[2].plot(trace_tissue2_mask.t, normalize(trace_tissue2_mask.y))
ax[2].plot(trace_tissue2.t, normalize(trace_tissue2.y))
ax[2].set_ylabel("tissue2")

ax[3].plot(trace_tissue3_mask.t, normalize(trace_tissue3_mask.y))
ax[3].plot(trace_tissue3.t, normalize(trace_tissue3.y))
ax[3].set_ylabel("tissue3")

fig.legend([l1, l2],["Masked", "Original"], loc="upper center", ncol=2)
plt.show()

Except for the fist tissue, we see that both versions are very similar, but the masked version seems a bit less noisy

In [ ]:
snr_mask = np.mean(trace_tissue1_mask.y) / np.std(trace_tissue1_mask.y)
snr_orig = np.mean(trace_tissue1.y) / np.std(trace_tissue1.y)

print(f"SNR (masked): {snr_mask}, SNR (original): {snr_orig}")